# Jet Bundles, Exterior Differential Systems & Intrinsic Geometric PDEs

**Theme: The PDE is the Manifold**


## 1. Introduction: The Geometric Turn

In previous episodes, we viewed a differential equation as an operator $\mathcal{L}$ acting on a function space. We asked: "Which function $u$ makes $\mathcal{L}u = 0$?"

In eps. 6, we invert this perspective. We realize that "derivatives" are not actions, but **independent coordinates**. A differential equation is not a problem to be solved, but a submanifold sitting inside a larger space called a **Jet Bundle**. Solving the equation means finding a surface that fits inside this submanifold while remaining tangent to a special "contact structure."

This geometric viewpoint allows us to handle overdetermined systems, classify singularities via algebra (Spencer Cohomology), and study equations that evolve the geometry of the universe itself (Ricci Flow).


## 2. Jet Bundles & Contact Geometry

To geometrise PDEs, we treat $x$, $u$, and $\partial u/\partial x$ as independent variables.

### 2.1 The 1-Jet Space ($J^1$)

Let $\pi: E \to M$ be a fiber bundle (e.g., $M$ is space, $E$ is the field). The **1-Jet Bundle** $J^1(E)$ has coordinates $(x, u, p)$, where $p$ represents the first derivative.

A "solution" is a section $u: M \to E$ that "lifts" to a surface in $J^1(E)$ given by $(x, u(x), u'(x))$.

### 2.2 The Contact Structure

Not every surface in $J^1(E)$ is a valid lift of a function. The variables must be compatible. This is enforced by the **Contact Form** (Cartan Distribution):

$$\theta = du - p \, dx = 0$$

A valid solution requires $\theta = 0$. Geometrically, this defines a field of hyperplanes (contact planes) at every point in $J^1(E)$. A solution is an **Integral Manifold** of this distribution—a surface that is everywhere tangent to these planes.

### 2.3 The PDE as a Submanifold

A first-order PDE $F(x, u, p) = 0$ is simply a hypersurface $\Sigma \subset J^1(E)$ defined by the zero set of $F$. Solving the PDE is equivalent to finding an integral manifold of the contact system that lies entirely inside $\Sigma$.


In [ ]:
# [The Contact Field. A 3D visualization of the 1-Jet space for a 1D function (x,u,p). The widget displays small "contact planes" at various points. A valid solution curve threads through these planes, while an invalid curve pierces them.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class ContactFieldWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['view_angle'] = ParameterSlider(
            name='view_angle',
            value=45.0,
            min_val=0.0,
            max_val=360.0,
            step=10.0,
            description='View Angle (degrees)'
        )
        self.sliders['show_solution'] = ParameterSlider(
            name='show_solution',
            value=1.0,
            min_val=0.0,
            max_val=1.0,
            step=1.0,
            description='Solution Type (0: Invalid, 1: Valid)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            angle = params['view_angle'] * np.pi / 180.0
            show_valid = params['show_solution'] > 0.5

                        # Jet space coordinates: x, u, p (p = du/dx)
            # Sample points in (x,u,p) space - increase density for better visualization
            x_vals = np.linspace(-2, 2, 5)
            u_vals = np.linspace(-2, 2, 5)
            p_vals = np.linspace(-2, 2, 5)
            X, U, P = np.meshgrid(x_vals, u_vals, p_vals)

            # Contact form: du - p dx = 0
            # Contact planes are spanned by vectors: v1 = (1, p, 0), v2 = (0, 0, 1)
            # We'll visualize these as filled, semi-transparent planes

            fig = self.plot_manager.create_plotly_figure(width=600, height=500)

            # Sample a subset of points for visualization (more points for better coverage)
            sample_indices = np.arange(0, len(X.flatten()), 3)
            x_sample = X.flatten()[sample_indices]
            u_sample = U.flatten()[sample_indices]
            p_sample = P.flatten()[sample_indices]

            # Draw contact planes as filled, visible surfaces
            plane_size = 0.4  # Larger planes for better visibility
            plane_resolution = 8  # Higher resolution for smoother planes

            for i in range(len(x_sample)):
                x0, u0, p0 = x_sample[i], u_sample[i], p_sample[i]

                # Contact plane at (x0, u0, p0): spanned by (1, p0, 0) and (0, 0, 1)
                # Create a plane patch with higher resolution
                s = np.linspace(-plane_size, plane_size, plane_resolution)
                t = np.linspace(-plane_size, plane_size, plane_resolution)
                S, T = np.meshgrid(s, t)

                # Plane coordinates: (x0 + s, u0 + p0*s + t, p0 + t)
                plane_x = x0 + S
                plane_u = u0 + p0 * S + T
                plane_p = p0 + T

                # Color based on position in jet space for better distinction
                color_intensity = (x0 + 2) / 4.0  # Normalize to [0, 1]

                fig.add_trace(go.Surface(
                    x=plane_x,
                    y=plane_u,
                    z=plane_p,
                    colorscale=[[0, 'rgba(100,150,255,0.6)'], [1, 'rgba(255,150,100,0.6)']],
                    surfacecolor=np.ones_like(plane_x) * color_intensity,
                    showscale=False,
                    opacity=0.7,
                    name='Contact Plane' if i == 0 else None,
                    showlegend=(i == 0)
                ))

            # Solution curve in jet space
            sol_x = np.linspace(-2, 2, 200)  # Higher resolution for smoother curve
            if show_valid:
                # Valid solution: u = x^2 / 2, so p = du/dx = x
                sol_u = sol_x**2 / 2.0
                sol_p = sol_x
                sol_title = 'Valid Solution (Tangent to Planes)'
                sol_color = '#00ff00'  # Bright green
            else:
                # Invalid: u = x^3 / 3, but p = x (not matching du/dx = x^2)
                sol_u = sol_x**3 / 3.0
                sol_p = sol_x  # Wrong: should be x^2
                sol_title = 'Invalid Curve (Pierces Planes)'
                sol_color = '#ff0000'  # Bright red

            # Add solution curve with thicker line and markers
            fig.add_trace(go.Scatter3d(
                x=sol_x,
                y=sol_u,
                z=sol_p,
                mode='lines+markers',
                line=dict(color=sol_color, width=8),
                marker=dict(size=3, color=sol_color),
                name=sol_title
            ))

            # Add start and end points for clarity
            fig.add_trace(go.Scatter3d(
                x=[sol_x[0], sol_x[-1]],
                y=[sol_u[0], sol_u[-1]],
                z=[sol_p[0], sol_p[-1]],
                mode='markers',
                marker=dict(size=10, color=sol_color, symbol='circle'),
                name='Endpoints',
                showlegend=False
            ))

            fig.update_layout(
                title='Contact Structure in 1-Jet Space $(x, u, p=du/dx)$<br>Contact Planes and Solution Curves',
                scene=dict(
                    xaxis_title='$x$',
                    yaxis_title='$u$',
                    zaxis_title='$p = du/dx$',
                    camera=dict(
                        eye=dict(
                            x=1.5 * np.cos(angle),
                            y=1.5 * np.sin(angle),
                            z=0.8
                        )
                    ),
                    aspectmode='cube',
                    bgcolor='rgba(0,0,0,0)',  # Transparent background for better contrast
                    xaxis=dict(backgroundcolor='rgba(0,0,0,0.1)'),
                    yaxis=dict(backgroundcolor='rgba(0,0,0,0.1)'),
                    zaxis=dict(backgroundcolor='rgba(0,0,0,0.1)')
                ),
                showlegend=True
            )

            fig.show()

widget = ContactFieldWidget(title="Contact Field in Jet Space")
widget.display()


## 3. Exterior Differential Systems (EDS)

Élie Cartan (1901) generalized this to **Exterior Differential Systems**, turning analysis into algebra.

### 3.1 The Differential Ideal

An EDS is a pair $(M, \mathcal{I})$, where $\mathcal{I}$ is a differential ideal of forms closed under exterior differentiation ($d\mathcal{I} \subset \mathcal{I}$).

A solution is a submanifold $\Sigma \subset M$ such that the pullback vanishes: $\iota^*\omega = 0$ for all $\omega \in \mathcal{I}$.

### 3.2 The Cartan-Kähler Theorem

How do we know if a solution exists? We calculate the **Cartan Characters** $s_0, s_1, \ldots$, which measure the "freedom" (dimension of the solution space) at each step of constructing the integral manifold.

**Cartan-Kähler Theorem:** If the system is real-analytic and "involutive" (algebraically consistent), local analytic solutions exist. This generalizes the Cauchy-Kovalevskaya theorem to geometric systems.


## 4. Geometric Evolution: Ricci Flow

We now apply these tools to the most famous geometric PDE: **Ricci Flow**, used by Perelman to prove the Poincaré Conjecture. Here, the unknown is not a function, but the **Riemannian Metric** $g_{ij}$ itself.

### 4.1 The Equation

We evolve the metric by its own curvature, smoothing out irregularities:

$$\frac{\partial g_{ij}}{\partial t} = -2 R_{ij}$$

where $R_{ij}$ is the Ricci curvature tensor. This is a nonlinear heat equation for geometry. Positive curvature regions shrink (spheres collapse); negative curvature regions expand.

### 4.2 Singularities & Surgery

The flow naturally forms singularities (e.g., a "neck" pinching off). To proceed, we must perform **Geometric Surgery**: cutting the manifold at the pinch point, capping the holes, and continuing the flow. This requires a precise understanding of the "canonical neighborhoods" of singularities.


In [10]:
# [Ricci Flow Surgery. A mesh of a "dumbbell" shape. Run the Ricci Flow simulation. Watch the neck pinch off (finite-time singularity). The widget pauses, performs "surgery" (topology change from 1 component to 2), and resumes the flow on the spheres.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq.widgets.base import AnimationWidget, ParameterSlider
from diffeq import PlotManager

class RicciFlowSurgeryWidget(AnimationWidget):
    def _setup_widgets(self):
        # Ensure is_playing exists (should be set by AnimationWidget.__init__, but ensure it)
        if not hasattr(self, 'is_playing'):
            self.is_playing = False

        # Call parent _setup_widgets (this initializes self.sliders dict)
        # Note: AnimationWidget doesn't override _setup_widgets, so this calls InteractiveWidget._setup_widgets
        # which just initializes self.sliders = {}
        super()._setup_widgets()

        # Now set up our sliders
        self.sliders['time'] = ParameterSlider(
            name='time',
            value=0.0,
            min_val=0.0,
            max_val=2.0,
            step=0.05,
            description='Flow Time $t$'
        )
        self.sliders['surgery'] = ParameterSlider(
            name='surgery',
            value=0.0,
            min_val=0.0,
            max_val=1.0,
            step=1.0,
            description='Perform Surgery (0: Off, 1: On)'
        )

        # Now set up animation controls (is_playing should be set by AnimationWidget.__init__)
        # But ensure it exists before calling _setup_animation_controls
        if not hasattr(self, 'is_playing'):
            self.is_playing = False
        self._setup_animation_controls()

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            t = params['time']
            do_surgery = params['surgery'] > 0.5

            # Dumbbell mesh: two spheres connected by cylinder
            # Parametric coordinates
            theta = np.linspace(0, 2*np.pi, 30)
            phi = np.linspace(0, np.pi, 30)
            THETA, PHI = np.meshgrid(theta, phi)

            # Left sphere (shrinks with time)
            sphere_radius = max(0.3, 1.0 - 0.3*t)
            X_left = -1.5 + sphere_radius * np.sin(PHI) * np.cos(THETA)
            Y_left = sphere_radius * np.sin(PHI) * np.sin(THETA)
            Z_left = sphere_radius * np.cos(PHI)

            # Right sphere (shrinks with time)
            X_right = 1.5 + sphere_radius * np.sin(PHI) * np.cos(THETA)
            Y_right = sphere_radius * np.sin(PHI) * np.sin(THETA)
            Z_right = sphere_radius * np.cos(PHI)

            # Neck: cylinder, but simulate pinch with t
            u = np.linspace(-0.5, 0.5, 20)
            v = np.linspace(0, 2*np.pi, 30)
            U, V = np.meshgrid(u, v)

            # Neck pinches at t ≈ 0.5
            if t < 0.5:
                neck_radius = max(0.05, 0.5 - t)
            else:
                neck_radius = 0.05  # Very thin

            X_neck = U
            Y_neck = neck_radius * np.cos(V)
            Z_neck = neck_radius * np.sin(V)

            # After surgery (t>1 and surgery enabled), separate spheres
            if do_surgery and t > 1.0:
                # Remove neck, show two separate spheres
                X_neck, Y_neck, Z_neck = np.array([]), np.array([]), np.array([])
                title = 'Post-Surgery: Two Spheres'
                status = 'Surgery Performed'
            elif t > 0.5 and neck_radius < 0.1:
                title = 'Pre-Surgery: Neck Pinching'
                status = 'Singularity Forming'
            else:
                title = 'Pre-Surgery: Dumbbell'
                status = 'Flow in Progress'

            fig = self.plot_manager.create_plotly_figure(width=600, height=500)

            # Plot left sphere
            fig.add_trace(go.Surface(
                x=X_left, y=Y_left, z=Z_left,
                colorscale='Blues',
                showscale=False,
                opacity=0.8
            ))

            # Plot right sphere
            fig.add_trace(go.Surface(
                x=X_right, y=Y_right, z=Z_right,
                colorscale='Reds',
                showscale=False,
                opacity=0.8
            ))

            # Plot neck if it exists
            if len(X_neck) > 0:
                fig.add_trace(go.Surface(
                    x=X_neck, y=Y_neck, z=Z_neck,
                    colorscale='Greys',
                    showscale=False,
                    opacity=0.6
                ))

            fig.update_layout(
                title=f'Ricci Flow on Dumbbell at $t={t:.2f}$<br>{title} - {status}',
                scene=dict(
                    xaxis_title='$X$',
                    yaxis_title='$Y$',
                    zaxis_title='$Z$',
                    aspectmode='data',
                    camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
                )
            )

            fig.show()

    def _animate(self):
        pass

widget = RicciFlowSurgeryWidget(title="Ricci Flow with Surgery")
widget.display()


## 5. Gauge Theory & Instantons

In particle physics, the "unknown" is a connection (vector potential $A$) on a principal bundle.

### 5.1 Yang-Mills Functional

We minimize the "curvature energy" of the connection:

$$S[A] = \int |F|^2 d^4x$$

where $F$ is the curvature (field strength). The Euler-Lagrange equations are the **Yang-Mills Equations**:

$$d_A^* F = 0$$

### 5.2 Self-Duality & Instantons

In 4 dimensions, we can solve a simpler, first-order condition called **Self-Duality**:

$$F = \star F$$

Solutions to this equation are called **Instantons**. They are topological solitons that tunnel between different vacua. Their existence is governed by topological invariants (Chern numbers), linking the PDE directly to the topology of the base manifold (Atiyah-Singer Index Theorem).



In [11]:
# [Instanton Visualization. A 3D projection of the 4D BPST Instanton. The widget visualizes the energy density concentration and the "twisting" of the gauge field at infinity (Chern number).]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class InstantonVisualizationWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['scale'] = ParameterSlider(
            name='scale',
            value=1.0,
            min_val=0.5,
            max_val=2.0,
            step=0.1,
            description='Instanton Scale $\\lambda$'
        )
        self.sliders['chern'] = ParameterSlider(
            name='chern',
            value=1.0,
            min_val=1.0,
            max_val=3.0,
            step=1.0,
            description='Chern Number $k$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            lam = params['scale']
            k = int(params['chern'])

            # BPST Instanton in 4D, project to 3D slice (e.g., x4=0)
            # Energy density ~ 1 / (r² + λ²)^4 for SU(2) instanton
            r = np.linspace(0.1, 5, 200)
            energy_density = 72.0 * lam**4 / (r**2 + lam**2)**4

            # 3D visualization: sphere with field lines twisting k times
            theta = np.linspace(0, 2*np.pi, 40)
            phi = np.linspace(0, np.pi, 40)
            THETA, PHI = np.meshgrid(theta, phi)

            # Sphere coordinates (S^3 projection to 3D)
            X = np.sin(PHI) * np.cos(THETA)
            Y = np.sin(PHI) * np.sin(THETA)
            Z = np.cos(PHI)

            # Twisting field: arrows with azimuthal twist k
            # Field direction on sphere
            U = -np.sin(PHI) * np.sin(k * THETA) * 0.3
            V = np.sin(PHI) * np.cos(k * THETA) * 0.3
            W = np.zeros_like(Z)

            fig = make_subplots(
                rows=1, cols=2,
                specs=[[{"type": "scatter3d"}, {"type": "xy"}]],
                subplot_titles=('Gauge Field on $S^3$ (Infinity)', 'Energy Density $|F|^2$'),
                horizontal_spacing=0.15
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=450,
                width=800,
                showlegend=True
            )

            # Left plot: 3D sphere with field lines
            # Sample points for cones (reduce density)
            skip = 5
            X_flat = X[::skip, ::skip].flatten()
            Y_flat = Y[::skip, ::skip].flatten()
            Z_flat = Z[::skip, ::skip].flatten()
            U_flat = U[::skip, ::skip].flatten()
            V_flat = V[::skip, ::skip].flatten()
            W_flat = W[::skip, ::skip].flatten()

            # Sphere surface
            fig.add_trace(
                go.Surface(
                    x=X*2, y=Y*2, z=Z*2,
                    colorscale='Viridis',
                    showscale=False,
                    opacity=0.6
                ),
                row=1, col=1
            )

            # Field vectors as cones
            fig.add_trace(
                go.Cone(
                    x=X_flat*2, y=Y_flat*2, z=Z_flat*2,
                    u=U_flat, v=V_flat, w=W_flat,
                    sizemode='scaled',
                    sizeref=0.3,
                    colorscale='Reds',
                    showscale=False
                ),
                row=1, col=1
            )

            # Right plot: Energy density
            fig.add_trace(
                go.Scatter(
                    x=r, y=energy_density,
                    mode='lines',
                    name='Energy Density $|F|^2$',
                    line=dict(color=self.plot_manager.config.colors[0], width=3)
                ),
                row=1, col=2
            )

            fig.update_layout(
                title=f'BPST Instanton (Scale $\\lambda={lam:.1f}$, Chern $k={k}$)<br>Energy Density and Gauge Twisting at Infinity',
                scene=dict(
                    xaxis_title='$X$',
                    yaxis_title='$Y$',
                    zaxis_title='$Z$',
                    aspectmode='cube',
                    camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
                )
            )

            fig.update_xaxes(title_text="$r$", row=1, col=2)
            fig.update_yaxes(title_text="$|F|^2$", type='log', row=1, col=2)

            # Note: scene2 is not a valid parameter for make_subplots with mixed types
            # The 2D plot uses standard x/y axes which are updated above

            fig.show()

widget = InstantonVisualizationWidget(title="BPST Instanton Visualization")
widget.display()


## 6. Fully Nonlinear Systems: Monge-Ampère

Finally, we tackle **Fully Nonlinear PDEs**, where the highest derivatives are wrapped in nonlinear functions.

### 6.1 The Monge-Ampère Equation

Arising in optimal transport and geometry, this equation prescribes the determinant of the Hessian:

$$\det(D^2 u) = f(x, u, \nabla u)$$

This equation is elliptic only if $u$ is **convex**. This introduces a strict geometric constraint on the *shape* of the solution, not just its smoothness.

### 6.2 Regularity & A Priori Estimates

Unlike linear equations, smoothness is not guaranteed even with smooth data. We rely on **A Priori Estimates** (Yau, 1978) to bound the derivatives independent of the solution, proving existence via the continuity method.



In [13]:
# [Optimal Transport map. A simulation of the Monge-Ampère equation solving the problem of morphing one probability density (a pile of sand) into another (a hole) with minimum cost. The potential u defines the transport map.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class OptimalTransportWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['interp'] = ParameterSlider(
            name='interp',
            value=0.5,
            min_val=0.0,
            max_val=1.0,
            step=0.1,
            description='Interpolation $t$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            t_interp = params['interp']

            # Source density: Gaussian at (0,0) - "pile of sand"
            # Target: Gaussian at (2,0) - "hole" (inverted)
            x = np.linspace(-3, 5, 80)
            y = np.linspace(-3, 3, 80)
            X, Y = np.meshgrid(x, y)

            # Source: pile of sand (Gaussian)
            source = np.exp(-((X)**2 + Y**2) / 0.5)
            source = source / np.sum(source)  # Normalize

            # Target: hole (Gaussian at different location)
            target = np.exp(-((X - 2.0)**2 + Y**2) / 0.5)
            target = target / np.sum(target)  # Normalize

            # Solve Monge-Ampère for potential u, transport map ∇u pushes source to target
            # Simplified: use optimal transport map T(x) = x + t * (target_center - source_center)
            # For Monge-Ampère: det(D²u) = source / target(T(x))
            # Here we approximate with a linear interpolation

            # Transport map: T(x) = x + t * shift
            shift_x = 2.0
            shift_y = 0.0

            # Interpolated positions
            X_trans = X + t_interp * shift_x
            Y_trans = Y + t_interp * shift_y

            # Interpolated density (push-forward of source)
            # Use inverse transport to sample target
            X_inv = X - t_interp * shift_x
            Y_inv = Y - t_interp * shift_y

            # Interpolated density: blend between source and target
            density_interp = (1 - t_interp) * source + t_interp * target

            fig = make_subplots(
                rows=1, cols=3,
                subplot_titles=('Source Density', f'Transport at $t={t_interp:.1f}$', 'Target Density'),
                horizontal_spacing=0.12
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=400,
                width=900,
                showlegend=False
            )

            # Source density
            fig.add_trace(
                go.Contour(
                    x=x, y=y, z=source,
                    colorscale='Blues',
                    showscale=True,
                    colorbar=dict(title="Density", x=0.28)
                ),
                row=1, col=1
            )

            # Interpolated (transport in progress)
            fig.add_trace(
                go.Contour(
                    x=x, y=y, z=density_interp,
                    colorscale='Greens',
                    showscale=True,
                    colorbar=dict(title="Density", x=0.64)
                ),
                row=1, col=2
            )

            # Target density
            fig.add_trace(
                go.Contour(
                    x=x, y=y, z=target,
                    colorscale='Reds',
                    showscale=True,
                    colorbar=dict(title="Density", x=1.0)
                ),
                row=1, col=3
            )

            # Transport arrows (sample)
            skip = 8
            arrow_x = X[::skip, ::skip].flatten()
            arrow_y = Y[::skip, ::skip].flatten()

            # Arrow directions
            dx = t_interp * shift_x
            dy = t_interp * shift_y

            # Add arrows to middle plot
            for i in range(len(arrow_x)):
                idx_x = int((arrow_x[i] + 3) * 80 / 8)
                idx_y = int((arrow_y[i] + 3) * 80 / 6)
                if 0 <= idx_x < 80 and 0 <= idx_y < 80 and source[idx_y, idx_x] > 0.01:
                    fig.add_trace(
                        go.Scatter(
                            x=[arrow_x[i], arrow_x[i] + dx],
                            y=[arrow_y[i], arrow_y[i] + dy],
                            mode='lines+markers',
                            line=dict(color='yellow', width=2),
                            marker=dict(size=4, symbol='arrow-right'),
                            showlegend=False
                        ),
                        row=1, col=2
                    )

            fig.update_xaxes(title_text="$x$", range=[-3, 5], row=1, col=1)
            fig.update_yaxes(title_text="$y$", range=[-3, 3], row=1, col=1)
            fig.update_xaxes(title_text="$x$", range=[-3, 5], row=1, col=2)
            fig.update_yaxes(title_text="$y$", range=[-3, 3], row=1, col=2)
            fig.update_xaxes(title_text="$x$", range=[-3, 5], row=1, col=3)
            fig.update_yaxes(title_text="$y$", range=[-3, 3], row=1, col=3)

            fig.update_layout(
                title=f'Optimal Transport via Monge-Ampère<br>Morphing Source to Target Density (Interpolation $t={t_interp:.1f}$)'
            )

            fig.show()

widget = OptimalTransportWidget(title="Optimal Transport Map via Monge-Ampère")
widget.display()


---

### References

* **Atiyah, M. F., Hitchin, N. J., & Singer, I. M.** (1978). *Self-duality in four-dimensional Riemannian geometry*.

* **Cartan, É.** (1901). *Sur l'intégration des systèmes d'équations aux différentielles totales*.

* **Ehresmann, C.** (1952). *Les connexions infinitésimales dans un espace fibré différentiable*.

* **Evans, L. C.** (1982). *Classical solutions of fully nonlinear, convex, second-order elliptic equations*.

* **Hamilton, R. S.** (1982). *Three-manifolds with positive Ricci curvature*.

* **Monge, G.** (1781). *Mémoire sur la théorie des déblais et des remblais*.

* **Perelman, G.** (2002). *The entropy formula for the Ricci flow and its geometric applications*.

* **Spencer, D. C.** (1965). *Deformation of structures on manifolds defined by transitive, continuous pseudogroups*.

* **Yau, S. T.** (1978). *On the Ricci curvature of a compact Kähler manifold and the complex Monge-Ampère equation*.
